# 量子ダイナミクスシミュレーション

第二回の後半は、量子コンピュータの並列性を利用した計算の代表例として、量子系のダイナミクスシミュレーションについて学びます。

```{contents} 目次
---
local: true
---
```

$\newcommand{ket}[1]{|#1\rangle}$

## 量子系のダイナミクスとは

量子力学について少しでも聞いたことのある方は、量子力学の根幹にシュレーディンガー方程式というものが存在することを知っているかと思います。この方程式は

$$
i \hbar \frac{\partial}{\partial t} \ket{\psi (t)} = H \ket{\psi (t)}
$$

などと表現され、時刻$t$のある系の状態$\ket{\psi (t)}$の時間微分（左辺）が$\ket{\psi (t)}$に作用するハミルトニアンという演算子で定まる（右辺）ということを表しています。ただしこの「微分形」の方程式は我々の目的には少し使いづらいので、ここでは等価な「積分形」にして

$$
\ket{\psi (t_1)} = \exp \left( -\frac{i}{\hbar} \int_{t_0}^{t_1} H dt \right) \ket{\psi (t_0)}
$$

と書いておきます。特に、ハミルトニアン$H$が直接時間に依存しない場合は

$$
\ket{\psi (t_1)} = \exp \left( -\frac{i}{\hbar} H(t_1 - t_0) \right) \ket{\psi (t_0)}
$$

です。量子状態に対する演算子（線形演算子）の指数関数もまた演算子なので、積分形のシュレーディンガー方程式は「$e^{-i/\hbar H(t_1-t_0)}$という演算子が系を時刻$t_0$の初期状態$\ket{\psi(t_0)}$から時刻$t_1$の状態$\ket{\psi(t_1)}$に発展させる」と読めます。さらに、定義上ハミルトニアンは「エルミート演算子」であり、それに虚数単位をかけて指数の冪にした$e^{-i/\hbar H t}$（以下これを時間発展演算子$U(t)$と呼びます）は「ユニタリ演算子」です。

ユニタリ演算子は量子計算の言葉で言えばゲートにあたります。したがって、ある量子系に関して、その初期状態を量子レジスタで表現でき、時間発展演算子を量子コンピュータの基本ゲートの組み合わせで実装できれば、その系のダイナミクス（＝時間発展）シミュレーションを量子コンピュータで行うことができます。

量子力学にあまり馴染みのない方にはここまでの議論がピンと来ないかもしれませんので、簡単な例を使って説明します。

空間中に固定されたスピン$\frac{1}{2}$原子核一つを考えます。ある方向（Z方向とします）のスピン$\pm \frac{1}{2}$の状態をそれぞれ$\ket{\uparrow}, \ket{\downarrow}$で表します。量子力学に馴染みのない方のための説明例で大いに量子力学的な概念を使っていますが、何の話かわからなければ「2つのケットで表現される、量子ビットのような物理系がある」と考えてください。量子ビットのような物理系なので、系の状態は一般に$\ket{\uparrow}$と$\ket{\downarrow}$の重ね合わせになります。時刻$t_0$で系が$\ket{\psi(t_0)} = \ket{\uparrow}$にあるとします。時刻$t_1$での系の状態を求めることは

$$
\ket{\psi (t_1)} = \alpha (t_1) \ket{\uparrow} + \beta (t_1) \ket{\downarrow}
$$

の$\alpha (t_1)$と$\beta (t_1)$を求めることに相当します。ここで$\alpha (t_0) = 1, \beta (t_0) = 0$です。

この原子核に$X$方向の一定磁場をかけます。この一定磁場によるハミルトニアンは定数$a$と演算子$\sigma_1$の積で表せます。ここで$\sigma_1$は$\ket{\uparrow}$と$\ket{\downarrow}$に対して

$$
\sigma_1 \ket{\uparrow} = \ket{\downarrow} \\
\sigma_1 \ket{\downarrow} = \ket{\uparrow}
$$

と作用します。時間発展演算子$U(t)$は[^hbar]

$$
U(t) = \exp (-i a \sigma_1 t) = I + (-i a t) \sigma_1 + \frac{1}{2} (-i a t)^2 \sigma_1^2 + \frac{1}{6} (-i a t)^3 \sigma_1^3 \cdots
$$

ですが（$I$は恒等演算子）、上の$\sigma_1$の定義からわかるように$\sigma_1^2 = I$なので

$$
\begin{align}
\exp (-i a \sigma_1 t) & = \left[ 1 + \frac{1}{2} (-i a t)^2 + \cdots \right] I + \left[(-i a t) + \frac{1}{6} (-i a t)^3 + \cdots \right] \sigma_1 \\
& = \cos(a t) I - i \sin(a t) \sigma_1
\end{align}
$$

と書けます。したがって、

$$
U(t) \ket{\psi (t_0)} = \exp (-i a \sigma_1 t) \ket{\uparrow} = \cos(a t) \ket{\uparrow} - i \sin(a t) \ket{\downarrow}
$$

です。


スピンが0でない原子核に磁場をかけると、スピンと磁場の方向が揃っていなければ磁場の方向を軸にスピンが「すりこぎ運動」をします。これはコマが重力中で起こす運動と同じで、核磁気共鳴（NMR、さらに医学応用のMRI）の原理に深く関わります。

[^hbar]: これ以降、本質と関係ないので定数$\hbar$を無視します。